# 🧬 StrandWeaver PathGNN Training on Google Colab

Train the **PathGNN** (Graph Neural Network for assembly path prediction) on a free T4 GPU.

**What this notebook does:**
1. Installs StrandWeaver + PyTorch + PyTorch Geometric
2. Generates synthetic training data (or loads uploaded data)
3. Converts edge/node CSV → PyG graph `Data` objects
4. Trains PathGNNModel (EdgeConv + GATv2 attention) with early stopping
5. Exports `pathgnn_model.pt` for download

**Runtime:** Set to **GPU** via `Runtime → Change runtime type → T4 GPU`

## 1. Setup & Installation

In [ ]:
# ── Verify GPU availability ──────────────────────────────────────────
import subprocess, sys
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                        capture_output=True, text=True)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
else:
    print("⚠ No GPU detected — training will be slower on CPU")

In [ ]:
# ── Install dependencies ─────────────────────────────────────────────
# PyTorch + PyG for the GNN, plus StrandWeaver itself
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q torch-geometric
!pip install -q xgboost scikit-learn

# Clone StrandWeaver and install
!git clone https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || echo "Already cloned"
!cd /content/strandweaver && pip install -e . --no-deps -q

print("\n✓ Installation complete")

In [ ]:
# ── Verify imports ────────────────────────────────────────────────────
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, MessagePassing
from torch_geometric.data import Data, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch {torch.__version__}  |  Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}  |  "
          f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 2. Upload or Generate Training Data

⚡ **Recommended: Option A (Upload)**  
Data generation is CPU-only and takes **hours** on Colab. Generate locally instead, then upload just the graph CSVs (~50 MB). The GPU is only used during training (Section 5).

**Option A:** Upload pre-generated graph CSVs from your local machine *(~1 min)*  
**Option B:** Generate fresh data on Colab *(~30 min per genome — NOT recommended)*

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OPTION A: Upload graph CSVs from local machine  (⚡ RECOMMENDED)
# ═══════════════════════════════════════════════════════════════════════
# Step 1 — On your LOCAL machine, package just the graph CSV files:
#
#   cd /path/to/strandweaver
#   find training_data_10x -name "*.csv" -path "*/graph_training/*" \
#       | tar czf graph_csvs.tar.gz -T -
#
#   (or from the original 20-genome set:)
#   find training_data -name "*.csv" -path "*/graph_training/*" \
#       | tar czf graph_csvs.tar.gz -T -
#
# Step 2 — Upload & extract here:

from google.colab import files
uploaded = files.upload()  # Upload graph_csvs.tar.gz

import os, tarfile
with tarfile.open("graph_csvs.tar.gz", "r:gz") as tar:
    tar.extractall("/content/")

# Auto-detect which directory was extracted
DATA_DIR = "/content/training_data_10x" if os.path.isdir("/content/training_data_10x") else "/content/training_data"
genome_dirs = sorted([
    os.path.join(root, "graph_training")
    for root, dirs, _ in os.walk(DATA_DIR)
    if "graph_training" in dirs
])
print(f"✓ Found {len(genome_dirs)} genome graph directories in {DATA_DIR}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OPTION B: Generate data on Colab  (⚠️ SLOW — NOT recommended)
# ═══════════════════════════════════════════════════════════════════════
# Data generation is CPU-only (~30 min/genome). Only use this if you
# cannot generate locally. Reduce -n to 3-5 for a quick test.
#
# GENERATE_ON_COLAB = False  # Flip to True to enable
# DATA_DIR = '/content/training_data'
#
# if GENERATE_ON_COLAB:
#     !cd /content/strandweaver && python -m strandweaver.cli train generate-data \
#         --genome-size 1000000 -n 5 \
#         --read-types hifi --read-types ont --read-types hic \
#         --coverage 30 --coverage 20 --coverage 15 \
#         --repeat-density 0.35 --graph-training --seed 42 \
#         -o {DATA_DIR}
#     print(f"\n✓ Training data generated in {DATA_DIR}")

## 3. Load Graph Data & Build PyG Datasets

In [ ]:
import csv
import json
import glob
import numpy as np
from pathlib import Path
from collections import defaultdict

# ── Feature columns (must match graph_training_data.py) ──────────────
EDGE_AI_FEATURES = [
    'overlap_length', 'overlap_identity', 'read1_length', 'read2_length',
    'coverage_r1', 'coverage_r2', 'gc_content_r1', 'gc_content_r2',
    'repeat_fraction_r1', 'repeat_fraction_r2',
    'kmer_diversity_r1', 'kmer_diversity_r2',
    'branching_factor_r1', 'branching_factor_r2',
    'hic_support', 'mapping_quality_r1', 'mapping_quality_r2',
]

NODE_FEATURES = [
    'coverage', 'gc_content', 'repeat_fraction', 'kmer_diversity',
    'branching_factor', 'hic_contact_density', 'allele_frequency',
    'heterozygosity', 'phase_consistency', 'mappability',
]

EDGE_LABEL_MAP = {'TRUE': 0, 'ALLELIC': 1, 'REPEAT': 2, 'SV_BREAK': 3, 'CHIMERIC': 4}
NODE_LABEL_MAP = {'HAP_A': 0, 'HAP_B': 1, 'BOTH': 2, 'REPEAT': 3, 'UNKNOWN': 4}

print(f"Edge features: {len(EDGE_AI_FEATURES)}")
print(f"Node features: {len(NODE_FEATURES)}")

In [ ]:
def load_graph_from_csvs(genome_dir: str) -> Data:
    """Load a single genome's graph data into a PyG Data object.

    Reads edge_ai CSV (edge features + labels) and diploid_ai CSV
    (node features + labels) and constructs a PyG graph.
    """
    graph_dir = Path(genome_dir) / 'graph_training'

    # ── Load edge data ────────────────────────────────────────────────
    edge_csv = list(graph_dir.glob('edge_ai_training_*.csv'))
    if not edge_csv:
        return None

    edge_features = []
    edge_labels = []
    with open(edge_csv[0]) as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                feat = [float(row[c]) for c in EDGE_AI_FEATURES]
                lbl = EDGE_LABEL_MAP.get(row['label'], 4)
                edge_features.append(feat)
                edge_labels.append(lbl)
            except (KeyError, ValueError):
                continue

    if len(edge_features) < 10:
        return None

    # ── Load node data ────────────────────────────────────────────────
    node_csv = list(graph_dir.glob('diploid_ai_training_*.csv'))
    node_features = []
    node_labels = []
    if node_csv:
        with open(node_csv[0]) as f:
            reader = csv.DictReader(f)
            for row in reader:
                try:
                    feat = [float(row[c]) for c in NODE_FEATURES]
                    lbl = NODE_LABEL_MAP.get(row.get('haplotype_label', 'UNKNOWN'), 4)
                    node_features.append(feat)
                    node_labels.append(lbl)
                except (KeyError, ValueError):
                    continue

    num_nodes = max(len(node_features), 100)

    # ── Build edge_index (synthetic: each edge row is an edge) ────────
    num_edges = len(edge_features)
    # Since we don't have explicit node IDs in the CSV, we create
    # synthetic edge indices mapping edges to node pairs
    src = np.random.RandomState(42).randint(0, num_nodes, size=num_edges)
    dst = np.random.RandomState(43).randint(0, num_nodes, size=num_edges)
    # Avoid self-loops
    mask = src != dst
    src, dst = src[mask], dst[mask]
    edge_features = [edge_features[i] for i in range(len(edge_features)) if i < len(mask) and mask[i]]
    edge_labels = [edge_labels[i] for i in range(len(edge_labels)) if i < len(mask) and mask[i]]

    # ── Build node features ──────────────────────────────────────────
    if len(node_features) < num_nodes:
        # Pad with zero features if needed
        while len(node_features) < num_nodes:
            node_features.append([0.0] * len(NODE_FEATURES))
            node_labels.append(4)  # UNKNOWN
    elif len(node_features) > num_nodes:
        node_features = node_features[:num_nodes]
        node_labels = node_labels[:num_nodes]

    # ── Convert to tensors ────────────────────────────────────────────
    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor([src.tolist(), dst.tolist()], dtype=torch.long)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)
    y_edge = torch.tensor(edge_labels, dtype=torch.long)
    y_node = torch.tensor(node_labels, dtype=torch.long)

    # Binary edge label: TRUE (0) vs everything else
    y_edge_binary = (y_edge == 0).float()

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y_edge=y_edge,
        y_edge_binary=y_edge_binary,
        y_node=y_node,
    )
    return data

print("Graph loader defined ✓")

In [ ]:
# ── Load all genome graphs ───────────────────────────────────────────
genome_dirs = sorted(glob.glob(f'{DATA_DIR}/genome_*'))
print(f"Found {len(genome_dirs)} genome directories")

all_graphs = []
for gd in genome_dirs:
    data = load_graph_from_csvs(gd)
    if data is not None:
        all_graphs.append(data)

print(f"Loaded {len(all_graphs)} graphs")
if all_graphs:
    g = all_graphs[0]
    print(f"  Example graph: {g.num_nodes} nodes, {g.num_edges} edges")
    print(f"  Node features: {g.x.shape}")
    print(f"  Edge features: {g.edge_attr.shape}")
    print(f"  Edge label dist: {dict(zip(*np.unique(g.y_edge.numpy(), return_counts=True)))}")

In [ ]:
# ── Train/val split ──────────────────────────────────────────────────
from sklearn.model_selection import train_test_split

train_graphs, val_graphs = train_test_split(
    all_graphs, test_size=0.2, random_state=42)

print(f"Training graphs: {len(train_graphs)}")
print(f"Validation graphs: {len(val_graphs)}")

## 4. Define PathGNN Model

In [ ]:
from dataclasses import dataclass
from typing import Dict, Optional

@dataclass
class GNNConfig:
    """Configuration for GNN model."""
    num_node_features: int = 10
    num_edge_features: int = 17
    hidden_channels: int = 64
    num_layers: int = 3
    dropout: float = 0.2
    learning_rate: float = 1e-3
    weight_decay: float = 1e-5
    batch_size: int = 1          # graph-level batching
    epochs: int = 100
    patience: int = 15
    use_attention: bool = True
    use_batch_norm: bool = True
    output_dim: int = 1
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'


class EdgeConvLayer(MessagePassing):
    """Custom edge convolution layer with edge features."""
    def __init__(self, in_channels, out_channels, edge_dim):
        super().__init__(aggr='mean')
        self.lin_src = nn.Linear(in_channels, out_channels)
        self.lin_dst = nn.Linear(in_channels, out_channels)
        self.lin_edge = nn.Linear(edge_dim, out_channels)
        self.lin_combined = nn.Linear(3 * out_channels, out_channels)

    def forward(self, x, edge_index, edge_attr=None):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr=None):
        msg_src = self.lin_src(x_i)
        msg_dst = self.lin_dst(x_j)
        if edge_attr is not None:
            msg_edge = self.lin_edge(edge_attr)
        else:
            msg_edge = torch.zeros_like(msg_src)
        msg = torch.cat([msg_src, msg_dst, msg_edge], dim=-1)
        return F.relu(self.lin_combined(msg))


class PathGNNModel(nn.Module):
    """Graph Neural Network for assembly path prediction.

    Architecture:
      - N EdgeConv layers with BatchNorm
      - Optional GATv2 attention layer
      - Edge classification head (TRUE vs non-TRUE)
      - Node classification head (haplotype assignment)
    """
    def __init__(self, config: GNNConfig):
        super().__init__()
        self.config = config
        self.device_name = config.device

        # Edge convolution layers
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        in_ch = config.num_node_features
        for _ in range(config.num_layers):
            self.convs.append(EdgeConvLayer(in_ch, config.hidden_channels, config.num_edge_features))
            if config.use_batch_norm:
                self.norms.append(nn.BatchNorm1d(config.hidden_channels))
            else:
                self.norms.append(nn.Identity())
            in_ch = config.hidden_channels

        # Attention
        if config.use_attention:
            self.attention = GATv2Conv(
                config.hidden_channels, config.hidden_channels // 2,
                heads=4, dropout=config.dropout)
            out_ch = config.hidden_channels * 2  # concat of 4 heads
        else:
            self.attention = None
            out_ch = config.hidden_channels

        # Prediction heads
        self.edge_head = nn.Sequential(
            nn.Linear(out_ch * 2, config.hidden_channels),
            nn.ReLU(), nn.Dropout(config.dropout),
            nn.Linear(config.hidden_channels, config.output_dim),
            nn.Sigmoid())

        self.node_head = nn.Sequential(
            nn.Linear(out_ch, config.hidden_channels),
            nn.ReLU(), nn.Dropout(config.dropout),
            nn.Linear(config.hidden_channels, 4),  # 4 haplotype classes
            nn.Softmax(dim=-1))

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr if hasattr(data, 'edge_attr') else None

        # Message passing
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index, edge_attr)
            x = norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.config.dropout, training=self.training)

        # Attention
        if self.attention is not None:
            x_attn = self.attention(x, edge_index)
            x = torch.cat([x, x_attn], dim=-1)

        # Edge predictions
        src, dst = edge_index
        edge_repr = torch.cat([x[src], x[dst]], dim=-1)
        edge_logits = self.edge_head(edge_repr).squeeze(-1)

        # Node predictions
        node_logits = self.node_head(x)

        return {'edge_logits': edge_logits, 'node_logits': node_logits, 'node_embeddings': x}


# Instantiate
config = GNNConfig()
model = PathGNNModel(config).to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"PathGNNModel: {total_params:,} parameters")
print(f"Device: {device}")

## 5. Training Loop

In [ ]:
import time
from sklearn.metrics import f1_score, accuracy_score

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Loss functions
edge_loss_fn = nn.BCELoss()   # Binary: TRUE edge vs not
node_loss_fn = nn.CrossEntropyLoss()  # 4-class haplotype


def train_epoch(model, graphs, optimizer):
    model.train()
    total_loss = 0
    total_edge_correct = 0
    total_edges = 0

    for data in graphs:
        data = data.to(device)
        optimizer.zero_grad()

        out = model(data)

        # Edge loss (binary: is this a TRUE edge?)
        e_loss = edge_loss_fn(out['edge_logits'], data.y_edge_binary.to(device))

        # Node loss (haplotype classification)
        # Only compute if we have node labels
        n_loss = torch.tensor(0.0, device=device)
        if hasattr(data, 'y_node'):
            valid_mask = data.y_node < 4  # exclude UNKNOWN
            if valid_mask.sum() > 0:
                n_loss = node_loss_fn(
                    out['node_logits'][valid_mask],
                    data.y_node[valid_mask].to(device))

        loss = e_loss + 0.3 * n_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        preds = (out['edge_logits'] > 0.5).long()
        total_edge_correct += (preds == data.y_edge_binary.long().to(device)).sum().item()
        total_edges += len(data.y_edge_binary)

    avg_loss = total_loss / len(graphs)
    accuracy = total_edge_correct / max(1, total_edges)
    return avg_loss, accuracy


@torch.no_grad()
def evaluate(model, graphs):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    for data in graphs:
        data = data.to(device)
        out = model(data)

        e_loss = edge_loss_fn(out['edge_logits'], data.y_edge_binary.to(device))
        total_loss += e_loss.item()

        preds = (out['edge_logits'] > 0.5).long().cpu().numpy()
        labels = data.y_edge_binary.long().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

    avg_loss = total_loss / max(1, len(graphs))
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    return avg_loss, acc, f1


print("Training functions defined ✓")

In [ ]:
# ── Train! ───────────────────────────────────────────────────────────
NUM_EPOCHS = config.epochs
PATIENCE = config.patience

best_val_loss = float('inf')
best_model_state = None
patience_counter = 0
history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

print(f"Training for up to {NUM_EPOCHS} epochs (patience={PATIENCE})")
print("=" * 70)

t_start = time.time()
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_acc = train_epoch(model, train_graphs, optimizer)
    val_loss, val_acc, val_f1 = evaluate(model, val_graphs)
    scheduler.step(val_loss)

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        patience_counter = 0
        marker = ' ★'
    else:
        patience_counter += 1
        marker = ''

    if epoch % 5 == 0 or epoch == 1 or marker:
        print(f"Epoch {epoch:3d}  "
              f"train_loss={train_loss:.4f}  "
              f"val_loss={val_loss:.4f}  "
              f"val_acc={val_acc:.4f}  "
              f"val_f1={val_f1:.4f}{marker}")

    if patience_counter >= PATIENCE:
        print(f"\nEarly stopping at epoch {epoch}")
        break

elapsed = time.time() - t_start
print(f"\nTraining complete in {elapsed:.1f}s")
print(f"Best val loss: {best_val_loss:.4f}")

# Restore best model
if best_model_state:
    model.load_state_dict(best_model_state)
    print("Restored best model checkpoint ✓")

## 6. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2, color='green')
ax2.plot(history['val_f1'], label='Val F1', linewidth=2, color='orange')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Score')
ax2.set_title('Validation Metrics')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/pathgnn_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved training curves ✓")

## 7. Export Trained Model

In [ ]:
# ── Save in StrandWeaver-compatible format ────────────────────────────
import os

OUTPUT_DIR = '/content/trained_pathgnn'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save the full model checkpoint
save_path = os.path.join(OUTPUT_DIR, 'pathgnn_model.pt')
torch.save({
    'model_type': 'PathGNNModel',
    'config': {
        'num_node_features': config.num_node_features,
        'num_edge_features': config.num_edge_features,
        'hidden_channels': config.hidden_channels,
        'num_layers': config.num_layers,
        'dropout': config.dropout,
        'use_attention': config.use_attention,
        'use_batch_norm': config.use_batch_norm,
        'output_dim': config.output_dim,
    },
    'model_state': model.state_dict(),
    'training_history': history,
    'best_val_loss': best_val_loss,
    'best_val_f1': max(history['val_f1']) if history['val_f1'] else 0,
}, save_path)

# Also save training metadata
meta = {
    'model_type': 'PathGNNModel',
    'num_parameters': sum(p.numel() for p in model.parameters()),
    'num_training_graphs': len(train_graphs),
    'num_validation_graphs': len(val_graphs),
    'best_val_loss': best_val_loss,
    'best_val_f1': max(history['val_f1']) if history['val_f1'] else 0,
    'epochs_trained': len(history['train_loss']),
    'config': vars(config),
}
with open(os.path.join(OUTPUT_DIR, 'training_metadata.json'), 'w') as f:
    json.dump(meta, f, indent=2, default=str)

print(f"Model saved to {save_path}")
print(f"Size: {os.path.getsize(save_path) / 1024:.1f} KB")
print(f"Parameters: {meta['num_parameters']:,}")

In [ ]:
# ── Download model files ─────────────────────────────────────────────
from google.colab import files

# Zip the output directory
!cd /content && tar czf pathgnn_trained.tar.gz trained_pathgnn/

print("\nDownloading trained model...")
print("After download, extract and copy to your StrandWeaver model dir:")
print("  tar xzf pathgnn_trained.tar.gz")
print("  cp trained_pathgnn/pathgnn_model.pt trained_models/pathgnn/pathgnn_model.pt")

files.download('/content/pathgnn_trained.tar.gz')

## 8. Quick Validation

Verify the exported model loads correctly in StrandWeaver's format.

In [ ]:
# ── Verify the model loads ────────────────────────────────────────────
checkpoint = torch.load(save_path, map_location='cpu')
print("Checkpoint keys:", list(checkpoint.keys()))
print(f"Config: {checkpoint['config']}")
print(f"State dict keys: {len(checkpoint['model_state'])} tensors")
print(f"Best val loss: {checkpoint['best_val_loss']:.4f}")
print(f"Best val F1: {checkpoint['best_val_f1']:.4f}")

# Reload into a fresh model
cfg2 = GNNConfig(**{k: v for k, v in checkpoint['config'].items() if k != 'device'})
model2 = PathGNNModel(cfg2)
model2.load_state_dict(checkpoint['model_state'])
model2.eval()
print("\n✓ Model reloads and is ready for inference")